# Chapter 11. Combining Data Sources

In many situtions, the information that you need will be spread across multiple data sets, so you will need to combine multiple sources. In this chapter, we explore how to combine information from multiple (tabular) data sets.

As a working example, we will use the baby names data collected by the Social Security Administration. Each data set in this collection contains all names of babies born in the United States in a particular year. This data is [publicly available](https://www.ssa.gov/OACT/babynames/limits.html), and a copy has been made available at `/data301/data/names/`.

In [ ]:
!ls /data301/data/names

# Chapter 11.1 Concatenating and Merging Data

## Concatenating

Sometimes, the _rows_ of data are spread across multiple files, and we want to combine the rows into a single data set. The process of combining rows from different data sets is known as **concatenation**.

For example, 

Let's concatenate the data from 2014 with the data from 2015. First, let's read in the two data sets.

In [ ]:
import pandas as pd
pd.options.display.max_rows = 8

names14 = pd.read_csv("/data301/data/names/yob2014.txt",
                      header=None,
                      names=["Name", "Gender", "Count"])
names14

In [ ]:
names15 = pd.read_csv("/data301/data/names/yob2015.txt",
                      header=None,
                      names=["Name", "Gender", "Count"])
names15

To concatenate the two, we can use the `pd.concat()` function, which accepts a _list_ of `pandas` objects (`DataFrames` or `Series`) and concatenates them.

In [ ]:
pd.concat([names14, names15])

There are two problems with the combined data set above. First, there is no longer any way to distinguish the 2014 data from the 2015 data. To fix this, we can add a "Year" column before we concatenate. Second, the indexes from the individual `DataFrame`s have been preserved. (To see this, observe that the last index in the `DataFrame` is 32,951, which corresponds to the number of rows in `names15`, but there are actually 66,128 rows in the `DataFrame`.) That means that there will be two rows with an index of 0, two rows with an index of 1, and so on. To force `pandas` to create a completely new index for this `DataFrame`, ignoring the indices from the individual `DataFrame`s, we specify `ignore_index=True`.

In [ ]:
names14["Year"] = 2014
names15["Year"] = 2015
names = pd.concat([names14, names15], ignore_index=True)
names

This is now a `DataFrame` that we can work with. Notice that the data is currently in "tidy" form, with one observation per row. It might be handy to show the number of babies born with each name and gender in the two years side-by-side, in "data cube" form. To do this, we can set the index and call `.unstack()`, just as we did in Chapter 2.

In [ ]:
names.set_index(["Name", "Gender", "Year"]).unstack()

## Merging (a.k.a. Joining)

More commonly, the data sets that we want to combine actually contain different information (i.e., variables) about the same observations. In other words, instead of combining the data sets row-wise, we want to combine them column-wise. The process of combining columns or variables from different data sets is known as **merging** or **joining**.

The observations in the two data sets may be in different orders, so merging is not as simple as stacking the `DataFrame`s side by side. Consider the following data sets, one which

|  Name  | Instrument |
|---------------------|
| Paul   |  bass      |
| Ringo  |  drums     |

In [ ]:
names14 = pd.read_csv("/data301/data/names/yob2014.txt",
                      header=None,
                      names=["Name", "Gender", "Count"])
names15 = pd.read_csv("/data301/data/names/yob2015.txt",
                      header=None,
                      names=["Name", "Gender", "Count"])

names14.merge(names15, on=["Name", "Gender"])

Notice that the variables we joined on appear once in the resulting `DataFrame`. The variable "Count", which we did not join on, appears twice---since there is "Count" in the first data set and a different "Count" in the second. Notice that `pandas` automatically appended the suffix `_x` to the name of the variable from the left data set and `_y` to the name from the right. We can customize the suffixes by specifying the `suffixes=` argument.

In [ ]:
names14.merge(names15, on=["Name", "Gender"], suffixes=("2014", "2015"))

In the code above, we assumed that the columns that we joined on had the same names in the two data sets. What if they had different names? For example, what if we want to match "Gender" in one data set to "Sex" in another? We can specify which variables to use from the left and right data sets using the `left_on=` and `right_on=` arguments.

In [ ]:
names14 = pd.read_csv("/data301/data/names/yob2014.txt",
                      header=None,
                      names=["Name", "Gender", "Count"])
names15 = pd.read_csv("/data301/data/names/yob2015.txt",
                      header=None,
                      names=["Name", "Sex", "Count"])

names14.merge(names15, left_on=("Name", "Gender"), right_on=("Name", "Sex"))

What if the "variables" that we want to join on are in the index? We can always call `.reset_index()` to make them columns, but we can also specify the arguments `left_index=True` or `right_index=True` to force `pandas` to use the index instead of columns.

In [ ]:
names14_indexed = names14.set_index(["Name", "Gender"])
names14_indexed

In [ ]:
names14_indexed.merge(names15, left_index=True, right_on=("Name", "Sex"))

Note that this worked because there were two levels in the index of the left `DataFrame` and also two variables to join on in the right `DataFrame`.

## One-to-One and Many-to-One Relationships

## Many-to-Many Relationships: A Cautionary Tale

In the baby names data, the name is not uniquely identifying. For example, there are both males and females with the name "Jessie". That is why we have to be sure to join on both name and gender. 

In [ ]:
names14 = pd.read_csv("/data301/data/names/yob2014.txt",
                      header=None,
                      names=["Name", "Gender", "Count"])
names15 = pd.read_csv("/data301/data/names/yob2015.txt",
                      header=None,
                      names=["Name", "Gender", "Count"])

names14[names14["Name"] == "Jessie"]

But what would go wrong if we joined on just "Name"? Let's try it and see.

In [ ]:
names14.merge(names15, on=["Name"])

We see that Emma ends up appearing four times.

- Female Emmas in 2014 are matched with female Emmas in 2015. (Good!)
- Male Emmas in 2014 are matched with male Emmas in 2015. (Good!)
- Female Emmas in 2014 are matched with male Emmas in 2015. (Huh?)
- Male Emmas in 2014 are matched with female Emmas in 2015. (Huh?)

If you are not careful about which variables you join on, you can end up with results that are unexpected and wrong.

# Exercises

**Exercise 1.** Make a line plot showing the popularity of your name over the years. How popular was your name in the year you were born? 

(If you have a rare name that does not appear in the data set, choose your friend's name.)

In [ ]:
# TYPE YOUR CODE HERE

Exercises 2-4 deal with the Movielens data (`/data301/data/ml-1m/`), which is a collection of movie ratings submitted by users. The information about the movies, ratings, and users are stored in three separate files. The column names are not included with the data files. You will need to read the data documentation (`/data301/data/ml-1m/README`) to figure out the column names and how the columns correspond across the data sets.

**Exercise 2.** Who's more generous with ratings: males or females? Calculate the average of the ratings given by male users, and the average of the ratings given by female users.

In [ ]:
# TYPE YOUR CODE HERE

**Exercise 3.** Among movies with at least 100 ratings, which movie had the highest average rating?

In [ ]:
# TYPE YOUR CODE HERE

**Exercise 4.** For each movie, calculate the average age of the users who rated it and the average rating. Make a scatterplot showing the relationship between age and rating, with each point representing a movie. (Optional: Use the size of each point to represent the number of users who rated the movie.)

In [ ]:
# TYPE YOUR CODE HERE